In [9]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../")

import scipy.io as sio
from motiflets.plotting import *
import matplotlib as mpl

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import warnings
warnings.simplefilter("ignore")

path = "../../motiflets_use_cases/chains/"

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [17]:
def read_penguin_data():
    series = pd.read_csv(path + "penguin.txt", 
                         names=(["X-Acc", "Y-Acc", "Z-Acc",
                                 "4", "5", "6",
                                 "7", "Pressure", "9"]),
                         delimiter="\t", header=None)
    ds_name = "Penguins (Longer Snippet)"
    return ds_name, series


def read_penguin_data_short():
    test = sio.loadmat(path + 'penguinshort.mat')
    series = pd.DataFrame(test["penguinshort"]).T
    ds_name = "Penguins (Snippet)"
    return ds_name, series


def test_plot_data():
    ds_name, series = read_penguin_data()
    series = series.iloc[497699 - 5000: 497699 + 5000, np.array([0, 7])].T

    ml = Motiflets(ds_name, series, elbow_deviation=1.25)
    ml.plot_dataset()

In [18]:
ds_name, series = read_penguin_data_short()

ml = Motiflets(
    ds_name, series,
    elbow_deviation=1,
    slack=0.8
)
# ml.plot_dataset()

k_max = 50
motif_length_range = np.arange(15, 50, 1)

_, all_minima = ml.fit_motif_length(
    k_max, motif_length_range,
    plot_elbows=False,
    plot_motifs_as_grid=False,
    plot_best_only=True,
    subsample=1)

# fig, ax = ml.plot_motifset()

TypeError: fit_motif_length() got an unexpected keyword argument 'plot_motifs_as_grid'

In [ ]:
@interact(motif_length = ml.motif_length_range[ml.all_extrema])
def show(motif_length=ml.motif_length):
    x = np.searchsorted(ml.all_extrema, motif_length)
    motif_length = ml.motif_length_range[x]
    candidates = np.zeros(len(ml.all_dists[x]), dtype=np.object)
    candidates[ml.all_elbows[x]] = ml.all_top_motiflets[x]  # need to unpack
    elbow_points = ml.all_elbows[x]

    # _plot_elbow_points(
    #    ml.ds_name, ml.data, motif_length, ml.all_elbows, candidates, ml.all_dists[x])

    plot_grid_motiflets(
        ml.ds_name, ml.series, 
        candidates, elbow_points,
        ml.all_dists[x], motif_length, 
        show_elbows=True,
        font_size=24,
        ground_truth=ml.ground_truth,
        dimension_labels=ml.dimension_labels)

In [ ]:
motif_length = 21
_ = ml.fit_k_elbow(
    k_max,
    motif_length=motif_length,  
    filter_duplicates=True,
    plot_elbows=True,
    plot_motifs_as_grid=False,
)

In [ ]:
@interact(k = widgets.IntSlider(min=2, max=k_max, step=1))
def show_elbows(k=ml.elbow_points[-1]):
    ml.plot_motifset(k)